
Predicting Fake News using NLP and Machine Learning 

Reperformance using Data set with fake news

https://towardsdatascience.com/predicting-fake-news-using-nlp-and-machine-learning-scikit-learn-glove-keras-lstm-7bbd557c3443


In [1]:
!pip install contractions

     |████████████████████████████████| 287 kB 13.3 MB/s 
     |████████████████████████████████| 106 kB 70.7 MB/s 


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.rcParams['figure.figsize'] = [10,10]
import seaborn as sns
sns.set_theme(style="darkgrid")
from wordcloud import WordCloud

import nltk
from nltk import sent_tokenize
''' Required for Google Colab'''
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
''' Required for Google Colab'''
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize

import contractions
import re
import itertools
import datetime
import time
from collections import Counter
import string

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.rcParams['figure.figsize'] = [10,10]
import seaborn as sns
sns.set_theme(style="darkgrid")

import nltk
from nltk import sent_tokenize
''' Required for Google Colab'''
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
''' Required for Google Colab'''
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
import contractions

import string
import joblib
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

# set seed
np.random.seed(42)

## based on validation data

In [30]:
train_df = pd.read_csv('https://raw.githubusercontent.com/smkerr/COVID-fake-news-detection/main/data/Constraint_Train.csv', header=0)
val_df = pd.read_csv('https://raw.githubusercontent.com/smkerr/COVID-fake-news-detection/main/data/Constraint_Val.csv', header=0)

In [31]:
stopwords = nltk.corpus.stopwords.words('english')

def cleantext(string):
    text = string.lower().split()
    text = " ".join(text)
    text = re.sub(r"http(\S)+",' ',text)    
    text = re.sub(r"www(\S)+",' ',text)
    text = re.sub(r"&",' and ',text)  
    tx = text.replace('&amp',' ')
    text = re.sub(r"[^a-zA-Z]+",' ',text)
    text = text.split()
    text = [w for w in text if not w in stopwords]
    text = " ".join(text)
    return text

In [32]:
train_df['tweet'] = train_df['tweet'].map(lambda x: cleantext(x))
val_df['tweet'] = val_df['tweet'].map(lambda x: cleantext(x))

In [33]:
#count_vectorizer = CountVectorizer(ngram_range=(1, 2))
#tf_idf_transformer = TfidfTransformer(smooth_idf=False)

count_vectorizer = CountVectorizer(ngram_range=(1, 1))
tf_idf_transformer = TfidfTransformer(smooth_idf=True)

# fit and transform train data to count vectorizer
count_vectorizer.fit(train_df['tweet'].values)
count_vect_train = count_vectorizer.transform(train_df['tweet'].values)

# fit the counts vector to tfidf transformer
tf_idf_transformer.fit(count_vect_train)
tf_idf_train = tf_idf_transformer.transform(count_vect_train)

# Transform the test data as well
count_vect_test = count_vectorizer.transform(val_df['tweet'].values)
tf_idf_test = tf_idf_transformer.transform(count_vect_test)

In [34]:
def get_perf_metrics(model, i):
    # model name
    model_name = type(model).__name__
    # time keeping
    start_time = time.time()
    print("Training {} model...".format(model_name))
    # Fitting of model
    model.fit(tf_idf_train, train_df["label"])
    print("Completed {} model training.".format(model_name))
    elapsed_time = time.time() - start_time
    # Time Elapsed
    print("Time elapsed: {:.2f} s.".format(elapsed_time))
    # Predictions
    y_pred = model.predict(tf_idf_test)
    # Add to ith row of dataframe - metrics
    df_perf_metrics.loc[i] = [
        model_name,
        model.score(tf_idf_train, train_df["label"]),
        model.score(tf_idf_test, val_df["label"]),
        precision_score(val_df["label"], y_pred, pos_label="real"),
        recall_score(val_df["label"], y_pred, pos_label="real"),
        f1_score(val_df["label"], y_pred, pos_label="real"), "{:.2f}".format(elapsed_time)
    ]
    # keep a track of trained models
    models_trained_list.append(model)
    print("Completed {} model's performance assessment.".format(model_name))

In [35]:
models_list = [LogisticRegression(),
               MultinomialNB(),
               RandomForestClassifier(),
               DecisionTreeClassifier(),
               GradientBoostingClassifier(),
               AdaBoostClassifier(),
               LinearSVC()]

In [36]:
df_perf_metrics = pd.DataFrame(columns=['Model', 'Accuracy_Training_Set', 'Accuracy_Test_Set', 'Precision', 'Recall', 'f1_score'])

In [37]:
df_perf_metrics = pd.DataFrame(columns=[
    'Model', 'Accuracy_Training_Set', 'Accuracy_Test_Set', 'Precision',
    'Recall', 'f1_score', 'Training Time (secs'
])

In [38]:
models_trained_list = []

In [39]:
%%time
for n, model in enumerate(models_list):
    get_perf_metrics(model, n)

Training LogisticRegression model...
Completed LogisticRegression model training.
Time elapsed: 0.13 s.
Completed LogisticRegression model's performance assessment.
Training MultinomialNB model...
Completed MultinomialNB model training.
Time elapsed: 0.02 s.
Completed MultinomialNB model's performance assessment.
Training RandomForestClassifier model...
Completed RandomForestClassifier model training.
Time elapsed: 3.04 s.
Completed RandomForestClassifier model's performance assessment.
Training DecisionTreeClassifier model...
Completed DecisionTreeClassifier model training.
Time elapsed: 0.85 s.
Completed DecisionTreeClassifier model's performance assessment.
Training GradientBoostingClassifier model...
Completed GradientBoostingClassifier model training.
Time elapsed: 2.97 s.
Completed GradientBoostingClassifier model's performance assessment.
Training AdaBoostClassifier model...
Completed AdaBoostClassifier model training.
Time elapsed: 0.97 s.
Completed AdaBoostClassifier model's p

## result

tweets incl. numbers:

In [29]:
df_perf_metrics

,Model,Accuracy_Training_Set,Accuracy_Test_Set,Precision,Recall,f1_score,Training Time (secs
0,LogisticRegression,0.959969,0.927570,0.943882,0.916071,0.929769,0.10
1,MultinomialNB,0.956231,0.912617,0.889074,0.951786,0.919362,0.02
2,RandomForestClassifier,1.000000,0.912617,0.930748,0.900000,0.915116,3.19
3,DecisionTreeClassifier,1.000000,0.853271,0.850435,0.873214,0.861674,0.78
4,GradientBoostingClassifier,0.899844,0.868224,0.904440,0.836607,0.869202,3.44
5,AdaBoostClassifier,0.882087,0.858411,0.878591,0.846429,0.862210,1.10
6,LinearSVC,0.999377,0.934579,0.947898,0.925893,0.936766,0.02


tweets excl. numbers:

In [40]:
df_perf_metrics

,Model,Accuracy_Training_Set,Accuracy_Test_Set,Precision,Recall,f1_score,Training Time (secs
0,LogisticRegression,0.958100,0.920561,0.932605,0.914286,0.923354,0.13
1,MultinomialNB,0.953427,0.908411,0.885000,0.948214,0.915517,0.02
2,RandomForestClassifier,0.999844,0.903738,0.927103,0.885714,0.905936,3.04
3,DecisionTreeClassifier,1.000000,0.838785,0.832618,0.866071,0.849015,0.85
4,GradientBoostingClassifier,0.892835,0.860748,0.901367,0.824107,0.861007,2.97
5,AdaBoostClassifier,0.867757,0.844393,0.858053,0.841964,0.849932,0.97
6,LinearSVC,0.997664,0.932710,0.938849,0.932143,0.935484,0.02
